In [1]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset,DataLoader
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt

In [2]:
torch.manual_seed(42)

In [3]:
df=pd.read_csv('./datasets/fashion-mnist_train.csv')
df.head(1)

,label,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,pixel9,...,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783,pixel784
0,2,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [4]:
device = torch.device("mps" if torch.backends.mps.is_available() else "cpu")
print(device)

mps


In [5]:
x=df.iloc[:,1:].values
y=df.iloc[:,0].values

In [6]:
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.2,random_state=42)

In [7]:
x_train=x_train/225.0
x_test=x_test/225.0

In [8]:
class customDataset(Dataset):
    def __init__(self,features,labels):
        self.features=torch.tensor(features,dtype=torch.float32).reshape(-1,1,28,28)
        self.labels=torch.tensor(labels,dtype=torch.long)
    def __len__(self):
        return self.features.shape[0]
    def __getitem__(self,ind):
        return self.features[ind],self.labels[ind]
    

In [9]:
train_dataset = customDataset(x_train,y_train)
test_dataset = customDataset(x_test,y_test)

In [10]:
test_dataset[0]

(tensor([[[0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
           0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
           0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
           0.0000, 0.0000, 0.0000, 0.0000],
          [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
           0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
           0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
           0.0000, 0.0000, 0.0000, 0.0000],
          [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
           0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
           0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
           0.0000, 0.0000, 0.0000, 0.0000],
          [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
           0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
           0.0000, 0.0000, 0.0000, 0.0000, 

In [11]:
train_dataloader = DataLoader(train_dataset,batch_size=32,shuffle=True,pin_memory=True)
test_dataloader= DataLoader(test_dataset,batch_size=32,shuffle=False,pin_memory=True)

In [31]:
class MyNN(nn.Module):
    def __init__(self,input_feature):
        super().__init__()
        self.features= nn.Sequential(
            nn.Conv2d(input_feature,32,kernel_size=3,padding='same'),
            nn.ReLU(),
            nn.BatchNorm2d(32),
            nn.MaxPool2d(kernel_size=2,stride=2),
            
            nn.Conv2d(32,64,kernel_size=3,padding='same'),
            nn.ReLU(),
            nn.BatchNorm2d(64),
            nn.MaxPool2d(kernel_size=2,stride=2)
            
        )
        self.classification = nn.Sequential(
            nn.Flatten(),
            nn.Linear(64*7*7,128),
            nn.ReLU(),
            nn.Dropout(p=0.4),

            nn.Linear(128,64),
            nn.ReLU(),
            nn.Dropout(p=0.4)
        )

    def forward(self,x):
        x= self.features(x)
        x= self.classification(x)
        return x

In [32]:
model=MyNN(1).to(device)

In [33]:
epochs=100
lr=0.1

In [34]:
citeration = nn.CrossEntropyLoss()

In [35]:
optimizer = optim.SGD(model.parameters(),lr=lr,weight_decay=1e-4)

In [36]:
for epoch in range(epochs):
    total_epoch_loss = 0
    for batch_features,batch_labels in train_dataloader:
        batch_features,batch_labels = batch_features.to(device),batch_labels.to(device)
        pred=model(batch_features)
        loss = citeration(pred,batch_labels)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        total_epoch_loss+=loss.item()
    avg_loss = total_epoch_loss/len(train_dataloader)
    print(f'Epoch: {epoch + 1} , Loss: {avg_loss}')
        

Epoch: 1 , Loss: 2.4343582031726836
Epoch: 2 , Loss: 2.1331076822280886
Epoch: 3 , Loss: 2.028107855439186
Epoch: 4 , Loss: 1.9599910276730856
Epoch: 5 , Loss: 1.955018118262291
Epoch: 6 , Loss: 1.9133216268221538
Epoch: 7 , Loss: 1.9040337035655974
Epoch: 8 , Loss: 1.897798668940862
Epoch: 9 , Loss: 1.8906999042431514
Epoch: 10 , Loss: 1.8950867652098338
Epoch: 11 , Loss: 1.8785371878147126
Epoch: 12 , Loss: 1.866512406349182
Epoch: 13 , Loss: 1.8766234058141709
Epoch: 14 , Loss: 1.8669428231318792
Epoch: 15 , Loss: 1.8474117159843444
Epoch: 16 , Loss: 1.854730988264084
Epoch: 17 , Loss: 1.8563255363702773
Epoch: 18 , Loss: 1.848164235830307
Epoch: 19 , Loss: 1.8323100214004517
Epoch: 20 , Loss: 1.823375139315923
Epoch: 21 , Loss: 1.8356380840539932
Epoch: 22 , Loss: 1.8231076658566794
Epoch: 23 , Loss: 1.8179310425917308
Epoch: 24 , Loss: 1.804753166079521
Epoch: 25 , Loss: 1.8020689582626024
Epoch: 26 , Loss: 1.7958900687297186
Epoch: 27 , Loss: 1.7864856855074565
Epoch: 28 , Loss: 

In [37]:
model.eval()

MyNN(
  (features): Sequential(
    (0): Conv2d(1, 32, kernel_size=(3, 3), stride=(1, 1), padding=same)
    (1): ReLU()
    (2): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (4): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=same)
    (5): ReLU()
    (6): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (7): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (classification): Sequential(
    (0): Flatten(start_dim=1, end_dim=-1)
    (1): Linear(in_features=3136, out_features=128, bias=True)
    (2): ReLU()
    (3): Dropout(p=0.4, inplace=False)
    (4): Linear(in_features=128, out_features=64, bias=True)
    (5): ReLU()
    (6): Dropout(p=0.4, inplace=False)
  )
)

In [38]:
correct=0
total=0
with torch.no_grad():
    for batch_features,batch_labels in test_dataloader : 
        batch_features,batch_labels= batch_features.to(device),batch_labels.to(device)
        pred= model(batch_features)
        _,predicted=torch.max(pred,1)
        total+=batch_labels.shape[0]
        correct+=(predicted==batch_labels).sum().item()
    print(correct/total)
    

0.9245
